In [18]:
import pandas as pd
import pygal
import numpy as np
from sklearn.cluster import DBSCAN

In [29]:
from IPython.display import display, HTML
base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [30]:
dataFrame = pd.read_json("data.json")
dataFrame.head()

,User,TimeStamp,Longitude,Latitude
0,Judy,2020-08-27 17:33:33,60.077519,13.988041
1,Ivan,2020-08-27 20:13:18,60.029391,13.903152
2,Judy,2020-08-27 18:22:23,60.078368,13.933152
3,Ivan,2020-08-27 03:38:36,60.002145,13.967506
4,James,2020-08-27 01:11:35,60.040521,13.966431


In [31]:
disp_dict = {}
for index, row in dataFrame.iterrows():
    if row['User'] not in disp_dict.keys():
        disp_dict[row['User']] = [(row['Latitude'], row['Longitude'])]
    else:
        disp_dict[row['User']].append((row['Latitude'], row['Longitude']))
xy_chart = pygal.XY(stroke=False)
[xy_chart.add(k,v) for k,v in sorted(disp_dict.items())]
display(HTML(base_html.format(rendered_chart=xy_chart.render(is_unicode=True))))


In [27]:
def getInfectedNames(dataFrame, inputName):
    assert (inputName in dataFrame['User'].tolist()), "User Doesn't exist"
    
    safe_distance = 0.0018288 #6 feets in kilometers
    
    model = DBSCAN(eps=safe_distance, min_samples=2, metric='haversine').fit(dataFrame[['Latitude', 'Longitude']])
    
    labels = model.labels_
    
    dataFrame['Cluster'] = model.labels_.tolist()
    
    inputNameClusters = set()
    for i in range(len(dataFrame)):
        if dataFrame['User'][i] == inputName:
            inputNameClusters.add(dataFrame['Cluster'][i])
               
    infected = set()
    for cluster in inputNameClusters:
        if cluster != -1: 
            namesInCluster = dataFrame.loc[dataFrame['Cluster'] == cluster, 'User'] 
            for i in range(len(namesInCluster)):
            
                name = namesInCluster.iloc[i]
                if name != inputName: 
                    infected.add(name)
    print("Potential infectious are:",*infected,sep="\n" )

In [28]:
getInfectedNames(dataFrame,"Carol")

Potential infectious are:
William
Ivan
James
